#  Use SARIMAX to model the time series process 

# Define global variable(train_end, predict_start, predict_end) and common functions

Define train and predict windows 

In [8]:
# for development 
# train_end='2019-6'
# predict_start='2019-7'
# predict_end='2019-9'

# for final submit 
train_end='2019-9'
predict_start='2019-10'
predict_end='2019-12'

# Import and process data 

## Import '../0_data/df_combined_fullsample.csv'

In [1]:
import pandas as pd
 #  contraceptive_logistics=pd.read_csv('../0_data/df_combined.csv') 
contraceptive_logistics=pd.read_csv('../0_data/df_combined_fullsample.csv')
contraceptive_logistics['day']=1
contraceptive_logistics['date']=pd.to_datetime(contraceptive_logistics[['year','month','day']])
contraceptive_logistics=contraceptive_logistics.sort_values('date')

# SARIMAX

- need to write this part 

# Distribute 

In [2]:
contraceptive_logistics.tail(3)

,date,site_code,product_code,year,month,stock_distributed,region,district,product_type,regionroads,...,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog,day
43603,2019-12-01,C3012,AS27134,2019,12,0.0,PORO-TCHOLOGO-BAGOUE,FERKESSEDOUGOU,Injectable Contraceptive,191.000000,...,1011.5000,30.20000,0.00000,5.200000,6.600000,12.700000,0.000000,0.000000,0.000000,1
42694,2019-12-01,C1701,AS27133,2019,12,0.0,AGNEBY-TIASSA-ME,TIASSALE,Injectable Contraceptive,98.000000,...,598.9047,78.26958,91.83673,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337,1
44503,2019-12-01,C5076,AS46000,2019,12,0.0,KABADOUGOU-BAFING-FOLON,KORO,NaN,97.381154,...,598.9047,78.26958,91.83673,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337,1


In [3]:
contraceptive_logistics['group_sum']=contraceptive_logistics.groupby(['product_code','district','date'])['stock_distributed'].transform('sum') 
contraceptive_logistics['share']=contraceptive_logistics['stock_distributed']/contraceptive_logistics['group_sum'] 
y=contraceptive_logistics.share.fillna(0)
y.shape

(44504,)

In [4]:
from sklearn.preprocessing import OneHotEncoder


In [5]:
columns_to_encode=['site_code', 'product_code', 'year', 'month',
       'region', 'district', 'product_type','site_type'] # categorical 
columns_continuous=['regionroads',
       'regionasphaltroads', 'regionearthroads', 'regionsurfacetreatmentroads',
       'regionpoorroads', 'poorroads', 'earthroads', 'asphaltroads', 'temp',
       'maxtemp', 'pressure', 'relativehumidity', 'rainfallsnowmelt',
       'visibility', 'windspeed', 'maxsustainedwindspeed', 'rainordrizzle',
       'storm', 'fog'] # continuous 

ohe = OneHotEncoder(sparse=False,categories='auto')
encoded_columns =  ohe.fit_transform(contraceptive_logistics[columns_to_encode].fillna('NA'))

 

features=pd.DataFrame(data=encoded_columns,columns=ohe.get_feature_names(columns_to_encode))
features[columns_continuous]=contraceptive_logistics[columns_continuous]

In [6]:
X=features
X.shape

(44504, 314)

## Train Test Split

In [9]:
train_end

X.index

contraceptive_logistics.shape

contraceptive_logistics.set_index('date')[:train_end].shape[0]

train_length=contraceptive_logistics.set_index('date')[:train_end].shape[0]

X_train=X[:train_length]
y_train=y[:train_length]

## Random Forest Model

In [10]:
X_train.columns[X_train.isna().any()].tolist()

[]

In [11]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=30,n_estimators=10,random_state=21)
regr.fit(X_train , y_train)
y_pred=regr.predict(X)
 

In [12]:
len(y_pred)

44504

## Export Distribute District Level Result  ../2_pipeline/final_distribute_districtlev.csv'

In [13]:
distribute_districtlev=contraceptive_logistics[['date','district','site_code','product_code','stock_distributed']].copy()
distribute_districtlev['pred_share_districtlev']=y_pred 
distribute_districtlev['agg_level']='district'
col_export_distribute_agg=['date','district','site_code','product_code','pred_share_districtlev','agg_level']
distribute_districtlev[col_export_distribute_agg].to_csv('../2_pipeline/final_distribute_districtlev.csv')

In [14]:
len(distribute_districtlev)

44504

In [15]:
distribute_districtlev.head()

,date,district,site_code,product_code,stock_distributed,pred_share_districtlev,agg_level
14324,2016-01-01,SIKENSI,C1098,AS27137,0.0,0.081555,district
26460,2016-01-01,TIEBISSOU,C2068,AS46000,0.0,0.073416,district
11574,2016-01-01,GRAND-BASSAM,C1087,AS27000,50.0,0.624580,district
34391,2016-01-01,BOUNA,C4018,AS27139,0.0,0.000000,district
32742,2016-01-01,BONDOUKOU,C4002,AS27132,0.0,0.500000,district
